### filename: run_replication.ipynb

### description: 
    last updated: 2018-04-24
        - update how we record outputs from regressions
            - separate IVs into "Central IVs" and "Controls"
            - collapse categorical variable dummies --> one variable (mean of abs(dummies))
    
    last updated: 2016-07-08
    Run models from an article for the purposes of comparing results to "manual/gold standard" replications by Julianna. 
    

### inputs:

### outputs:

## TO-DOs:
    2018-05-09
        collapse all DVs into 1 regression?
    
    
@author: Misha


In [1]:
# the article we are running
ARTICLE_ID = 2753

In [2]:
from __future__ import division
import pandas as pd
import pickle
import sys
sys.path.append('../')    
import GSSUtility as GU
import numpy as np
import statsmodels.formula.api as smf 
import random
from scipy.stats import pearsonr, ttest_ind, ttest_rel
import time
from collections import Counter
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sb
custom_style = {'axes.facecolor': 'white',
                'grid.color': '0.15',
                'grid.linestyle':'-.'}
sb.set_style("darkgrid", rc=custom_style)

%rm ../GSSUtility.pyc # remove this file because otherwise it will be used instead of the updated .py file
reload(GU)

pathToData = '../../Data/'
dataCont = GU.dataContainer(pathToData)

Loading DataFrame df. This may take a few minutes.


In [4]:
################################
# ANALYSIS

articlesToUse = GU.filterArticles(dataCont.articleClasses, GSSYearsUsed=True, GSSYearsPossible=False, \
                                    centralIVs=False, nextYearBound=0, linearModels=False)            
article = [a for a in articlesToUse if a.articleID == ARTICLE_ID][0]
maxYearUsed = max(article.GSSYearsUsed)

print 'article id:', article.articleID
print
print 'GSS Years Used:', article.GSSYearsUsed
print 'GSS Years Possible:', article.GSSYearsPossible
print 'DVs:', article.DVs
print
print 'IVs:', article.IVs
print
print 'Controls:', article.controls
print
print 'Central IVs:', article.centralIVs

article id: 2753

GSS Years Used: [1977, 1978, 1980, 1982, 1983, 1984, 1985, 1987, 1988]
GSS Years Possible: [1989, 1990, 1991, 1993, 1994, 2004, 2006, 2008, 2010, 2012, 2014, 2016]
DVs: ['ABANY', 'ABDEFECT', 'ABPOOR', 'ABNOMORE', 'ABHLTH', 'ABRAPE', 'ABSINGLE']

IVs: ['SPREL', 'RELIG', 'RELIG16', 'AGE', 'MARITAL', 'CHILDS', 'RACE', 'DENOM', 'OTHER', 'DENOM16', 'OTH16', 'DEGREE']

Controls: []

Central IVs: ['SPREL', 'RELIG', 'RELIG16', 'MARITAL', 'DENOM', 'OTHER', 'DENOM16', 'OTH16']


In [5]:
# define the storage containers for outputs
outcomes = ['propSig_ControlVars', 'paramSizesNormed_ControlVars', 'Rs', 'adjRs', 
            'propSig_CentralVars', 'paramSizesNormed_CentralVars']

output = defaultdict(list)
for outcome in outcomes:
    output[outcome] = []

RHS = list(set(article.IVs + article.controls))
dfoutput = pd.DataFrame(index=article.DVs)

In [6]:
print 'Running article:', article.articleID, 'on', maxYearUsed

for DV in article.DVs:
    print DV, '~', RHS
    #     RHS.remove('AGEWED')

#         futureYearsPossible = [yr for yr in article.GSSYearsPossible if yr > maxYearUsed]
#         nextYear = min(futureYearsPossible) # the arguments of GU.filterArticles function ensure that there is a suitable future year (within bound)

#             log.write('id'+str(article.articleID)+' year '+str(maxYearUsed))

    res = GU.runModel(dataCont, maxYearUsed, DV, RHS) #, 
#                                 custom_data=custom_data,
#                                 standardized=False) # models run on max year of data used
    
    if not res: 
        continue
    
#     # TREATMENT OF DUMMY VARIABLES
#     # If #(dummies) for a variable is > 1, then take the absolute value of coefficients of those dummies and avg them
#     # if #(dummies) == 1 or variable is continuous, do nothing
#     def f(x):
#         return x.abs().mean() if len(x) > 1 else x[0]
#     res.params = res.params.groupby(lambda x: x.split('[')[0].split(',')[0]).apply(f)
    
    
#     res.params = res.params.abs().groupby(lambda x: x.split('[')[0].split(',')[0]).mean()
#     res.pvalues = res.pvalues.groupby(lambda x: x.split('[')[0].split(',')[0]).mean()

    dfoutput.loc[DV, 'Rs'] = res.rsquared
    dfoutput.loc[DV, 'adjRs'] = res.rsquared_adj

    for col in res.params.index:          
        dfoutput.loc[DV, col] = res.params[col]
        
    break


Running article: 2753 on 1988
ABANY ~ ['RELIG', 'CHILDS', 'DENOM16', 'DEGREE', 'AGE', 'RELIG16', 'OTHER', 'MARITAL', 'SPREL', 'DENOM', 'OTH16', 'RACE']
# rows before dropna(): 1481
Dropping column RELIG because it is constant
Dropping column DENOM16 because it is constant
Dropping column RELIG16 because it is constant
Dropping column MARITAL because it is constant
Dropping column DENOM because it is constant
# rows after dropna(): 53


In [7]:
GU.runModel(dataCont, maxYearUsed, DV, RHS).summary()

# rows before dropna(): 1481
Dropping column RELIG because it is constant
Dropping column DENOM16 because it is constant
Dropping column RELIG16 because it is constant
Dropping column MARITAL because it is constant
Dropping column DENOM because it is constant
# rows after dropna(): 53


<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                                
======================================================================================
Dep. Variable:     standardize(ABANY, ddof=1)   R-squared:                       0.084
Model:                                    OLS   Adj. R-squared:                 -0.035
Method:                         Least Squares   F-statistic:                    0.7071
Date:                        Tue, 22 May 2018   Prob (F-statistic):              0.645
Time:                                23:26:26   Log-Likelihood:                -72.361
No. Observations:                          53   AIC:                             158.7
Df Residuals:                              46   BIC:                             172.5
Df Model:                                   6                                         
Covariance Type:                    nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                      -0.0086      0.155     -0.055      0.956      -0.321       0.303
C(SPREL)[T.4.0]                 0.2644      0.755      0.350      0.728      -1.256       1.785
C(RACE)[T.2.0]                 -0.3074      0.552     -0.557      0.580      -1.418       0.803
C(RACE)[T.3.0]                  0.3847      0.643      0.599      0.552      -0.909       1.678
standardize(CHILDS, ddof=1)    -0.1774      0.189     -0.937      0.354      -0.558       0.204
standardize(DEGREE, ddof=1)    -0.2354      0.162     -1.454      0.153      -0.561       0.091
standardize(AGE, ddof=1)       -0.0932      0.180     -0.518      0.607      -0.456       0.269
==============================================================================
Omnibus:                       14.822   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               17.420
Skew:                          -1.391   Prob(JB):                     0.000165
Kurtosis:                       3.389   Cond. No.                         7.44
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [215]:
design = dataCont.df.loc[1990, ['HEALTH'] + RHS]
design

,HEALTH,MEMFARM,MEMUNION,MEMOTHER,MEMPROF,AGE,EDUC,MEMSCHL,MEMFRAT,MEMSERV,MEMVET,MEMSPORT,MEMNAT,INCOME,MEMLIT,MEMHOBBY,SEX,MEMYOUTH,MEMGREEK,MEMPOLIT
year,,,,,,,,,,,,,,,,,,,,
1990,NaN,2.0,2.0,1.0,2.0,65.0,16.0,2.0,2.0,2.0,2.0,2.0,1.0,12.0,2.0,2.0,2,2.0,2.0,2.0
1990,NaN,2.0,2.0,1.0,2.0,42.0,20.0,2.0,2.0,2.0,2.0,2.0,2.0,12.0,2.0,2.0,1,2.0,2.0,2.0
1990,1.0,NaN,NaN,NaN,NaN,25.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
1990,1.0,NaN,NaN,NaN,NaN,39.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,2,NaN,NaN,NaN
1990,NaN,2.0,2.0,1.0,2.0,55.0,8.0,2.0,2.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,1,2.0,2.0,2.0
1990,2.0,NaN,NaN,NaN,NaN,82.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,2,NaN,NaN,NaN
1990,NaN,2.0,2.0,2.0,2.0,54.0,19.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,2.0,2,2.0,2.0,2.0
1990,4.0,NaN,NaN,NaN,NaN,61.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,1,NaN,NaN,NaN
1990,3.0,NaN,NaN,NaN,NaN,53.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,2,NaN,NaN,NaN


In [229]:
# design.dropna(axis=1, thresh=int(0.2*len(design)))

In [146]:
#2506
# dfoutput.loc[['SPKRAC', 'ANTIREL', 'LIBHOMO', 'COLRAC'], :].mean().T
# MARITAL AVG = [6:10].mean()
# race avg = [3:5].mean()
# dfoutput.loc[['FEPOLY', 'FEPRES', 'FEFAM'], :].mean().T
# dfoutput.loc[['HOMOSEX', 'PREMARSX', 'XMARSEX'], :].mean().T
# dfoutput.loc[['NATENVIR','NATHEAL','NATEDUC', 'NATSOC'], :].mean().T
# dfoutput.loc[['NATRACE', 'NATCITY', 'NATFARE'], :].mean().T
# dfoutput.loc[['COMMUN', 'CHINA', 'RUSSIA'], :].mean().T[3:5].mean()

In [232]:
design = GU.df.loc[1976, [DV] + RHS]

nominals = GU.createFormula(dataCont, design, return_nominals=True)
non_nominals = list(set(design.columns[1:]) - set(nominals)) # list because sets are unhashable and cant be used for indices and [1:] 

print nominals
print non_nominals

['EVWORK', 'UNION', 'SEX', 'RACE', 'UNEMP']
['PRESTIGE', 'DEGREE', 'WRKSTAT', 'PAPRES16', 'MAEDUC', 'PAEDUC', 'HRS1', 'INDUSTRY', 'EDUC', 'AGE']


(1499, 5)

In [243]:
print len(design)

if len(non_nominals)>0: 
    design[non_nominals] = design[non_nominals].fillna(design[non_nominals].mean()) # the naive way
if len(nominals)>0:
    design[nominals] = design[nominals].fillna(design[nominals].mode().iloc[0]) # iloc because 
    
len(design.dropna())

1499


828

In [244]:
design

Index([u'RINCOME'], dtype='object')

In [135]:
dfoutput.to_csv('%d_output.csv' % ARTICLE_ID)

In [270]:
from scipy.stats import pearsonr
pearsonr([0.359, 0.469], [0.215, 1.166])

(1.0, 0.0)

In [271]:
np.mean([0.0089, 0.0124])

0.01065